In [1]:
##Vanishing Gradient-during backprop the error gradient is taken into account wrt each parameter and each learnable parameter is updated 
##accordingly..but at times..during backprop..the gradient keeps decreasing...and might even  fall to zero..due to which gradient descent update in the lower
##layers leave them virtually unchanged

##Exploding Gradient-Ulta of vanishing gradient....gradient keeps on increasing..weight updates become erratic and the model diverges 

In [2]:
##Solution-We want the signal to flow freely in both the directions...in fwd dir while making predictions and in reverse direction while backprop
##FOR THE SIGNAL TO FLOW PROPERLY:
##1.variance of op @each layer ==varaiance of ip@each layer
##2.variance of gradients @ each layer should be same while flowing through a layer @ reverse
##BUT THE ABOVE IS NOT POSSIBLE UNLESS THE LAYER HAS AN EQUAL NO. OF INPUTS(fan-in) AND NEURONS(fan-out)
##COMPROMISE-weights are initialized randomly  based on Glorot inititalization


In [3]:
##GLOROT INITIALIZATION/XAVIER INITIALIZATION=
##Normal distribution with mean 0 and variance sigma_2=1/fan_avg
##uniform distribution between -r and +r,with r=sqrt(3/fan_avg)

##Above..if we replace fan_avg with fan_in,we get LECUN Initialization(USED WITH SELU)--->scaled exponential linear unit


##INITIALIZATION      ACTIVATION FUNCTIONS               variance
##GLOROT              none,tanh,logistic,sftmax          1/fan_avg
##He                  Relu(+the variants)                 2/fan_in
##LeCun               SELU                                 1/fan_in

##BY DEFAUL KERAS USES GLOROT INITIALIZATION WITH UNIFORM DISTRIBUTION

## we can change this to He initialization via kernel_initializer

#tf.keras.layers.Dense(10,activation="relu",kernel_initializer="he_normal")

In [ ]:
#Problem with Relu


# they might be suffering from Dying Relu.It occurs due to a large lr when the weights get tweaked in such a way thta 
# weighted sum of the inputs are negative for all the instances in the training set.(cuz for relu gradient of negative is 0)

##SOLUTION:Leaky Relu
Lrelu=max(alpha*z,z)
alpha=how much the function leaks : tis the slope when z<0

##Randomized Leaky Relu--->alpha is picked randomly during training and fixed to an average during testing

##Parametric Leaky Relu--->alpha is learnable

##ELU(z)={alpha(e**z-1) if z<0}
##       {z if            z>=0}

In [4]:
#heirarchy of activation functions

#SELU>ELU>Leaky Relu>Relu>tanh>logistic

In [ ]:
##TO use leaky Relu
leaky_relu=tf.keras.layers.LeakyReLU(alpha=0.2)
layer=tf.keras.layers.Dense(10,activation=leaky_relu,kernel_initializer="he_normal")

##FOR SELU
layer=tf.keras.layers.Dense(10,activation="selu",kernerl_initializer="lecun_normal")

In [ ]:
#BATCH NORMALIZATION

introduced as a solution to vanisihing/exploding gradients

what it does and where is it applied

-before or after the activation function

-a.zero centering and normalizing each input

-b.scaling and shifting the result using two new parameter vectors

NOTE:a and b is done per batch

In [ ]:
##NORMALIZATION IS BASICALLY---->centering the batch and normalizing them,then scaling and shifting them
##the centering is done by substracting the batch_mean from the ip
##the normalizing is done by dividing the above term by the sqrt(variance+epsilon)....epsilon is a small number..present there to prevent division by zero

In [1]:
##The output_scale vector  and the output_offset vector are learnable
##NOTE--------------------------------------------------------------
##the final ip_mean vector and ip_sd_vector are estimated during training,but they are not used at all during training,only after training

In [2]:
##One of the important hyperparameter of BatchNormalization is momentum(v)
##This is used when updating the exponential moving averages
##given a new 'v',(a new vector of input means or standard deviations computed over the current batch),running average V is updated via
##V<--V*momentum+v(1-momentum)

In [3]:
##another important parameter is axis:determines which axis should be normalized
##defaults to -1,i.e the last axis
##Eg..2d ip batch-->[batch_size,features]---->axis=-1--->the features will be normalized

##but if batchnormalizatoin is done before flatten,ip batch-->3d-->[batch_size,height,weight]
##

In [ ]:
##GRADIENT CLIPPING

optimizer=tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse",optimizer=optimizer)

this will clip every component of the gradient vector to a value -1.0 and 1.0 if it is not within the threshold

clipvalue will change the orientation of the graient vector...better use clipnorm(preserves orientation)-->clips the whole gradient if 
l2 norm is greater than the threshold i picked.

In [ ]:
##MOMENTUM OPTIMIZATION --->takes into account the value of the previous gradient...unlike plain gradient descent
##Tis like dis-->m<-Bm-ndj(0)/d0............ndj(0)/d0-->local gradient;m=momentum vector
#--------------->0<-0+m
#Hyparameters involved--->momentum:set between 0(high friction) and 1(no friction)--->typical_value=0.9

##USAGE:
##optimizer=tf.keras.optimizers.SGD(lr=0.001,momentum=0.9)

Nesterov Acclerated Gradient==>better than vanilla moment====>only diff===> 

m<-Bm-ndj(0+Bm)/d0

it measures the gradient slightly ahead in the direction of the momentum

In [ ]:

##USING RMS PROP==>
optimizer=tf.keras.optimizers.RMSprop(lr=0.001,rho=0.9)

##USING ADAM
optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999)


In [4]:
##LEARINNG RATE SCHEDULER

In [ ]:
##power_scheduling--->r/2,r/3,r/4--->after every  s steps
optimizer=keras.optimizers.SGD(lr=0.01,decay=1e-4)



In [ ]:
##exponential scheduling

def exponential_decay(lr0,s):
  def exponential_decay_fn(epoch):
    return 0.01*0.1**(epoch/20)
  return exponential_decay_fn

exponential_decay_fn=exponential_decay(lr=0.01,s=20)

lr_scheduler=keras.callbacks.LearningRateScheduler(exponential_decay_fn)

##pass the scheduler to the callback option in fit option


##pieceWise Scheduler

def pw_c(epoch):
  if epoch<5:
    return 0.01
  elif epoch<15:
    return 0.005
  else:
    return 0.001

pw_scheduler=keras.callbacks.LearningRateScheduler(pw_c)
##SAME AS exponential scheduling

##NOTE-------->IF OUR SCHEDULER USES EPOCH as input,we need to set the initial_epoch to proper value,if we are loading from a saved model that had lr_scheduler ip as epoch
##otherwise the weights will get wrecked




##Performance Scheduling

lr_scheduler=keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=5)

##if the best validation loss doesnt improve over 5 consecutive epochs,it will multiply the lr by 0.5



##UPDATE THE LEARNING RATE AT EVERY S step instead of epoch

s=20*len(X_train)
lr=keras.optimizers.schedules.ExponentialDecay(0.01,s,0.1)

opimizer=keras.optimizers.SGD(lr)
